In [1]:
import numpy as np
import pandas as pd
from typing import Dict
from constants import *
import random

In [3]:
pd.Series(np.zeros(100))

0     0.0
1     0.0
2     0.0
3     0.0
4     0.0
     ... 
95    0.0
96    0.0
97    0.0
98    0.0
99    0.0
Length: 100, dtype: float64

In [3]:
def mod_beta_random(low: float, high: float, mean: float, std: float, samples: int) -> np.ndarray:
    """
    Generate random numbers from a transformed Beta distribution
    Formulas to derive parameters are taken from here
    https://stats.stackexchange.com/questions/12232/calculating-the-parameters-of-a-beta-distribution-using-the-mean-and-variance
    :param low: the lowest value possible
    :param high: the highest value possible
    :param mean: the mean of the population
    :param std: the standard deviation of the population
    :param samples: the number of samples to generate, or a tuple of the output shape
    :return: a np.ndarray of size "shape" drawn from the distribution
    >>> beta = mod_beta_random(0., 10., 4., 1.9, 500)
    >>> np.all((0 <= beta) & (beta <= 10))
    True
    >>> np.all((0.5 <= beta) & (beta <= 9.5))
    False
    >>> mu = beta.mean()
    >>> 3.8 < mu and mu < 4.2
    True
    >>> std = beta.std()
    >>> 1.8 < std and std < 2.0
    True
    """
    assert low <= mean <= high, "Population mean out of bound."
    mu = (mean - low) / (high - low)
    sigma = std / (high - low)
    assert sigma ** 2 <= mu * (1 - mu), "Population standard deviation too large for a Beta distribution to exist."
    a = ((1 - mu) / (sigma ** 2) - 1 / mu) * (mu ** 2)
    b = a * (1 / mu - 1)
    beta = np.random.beta(a, b, samples)
    beta = low + beta * (high - low)
    return beta

In [4]:
class Food:
    def __init__(self, inventory=None):
        """
        Initialize a Food object which is either empty, or based on a dataframe or total pounds of food.
        :param inventory:
        >>> Food().df
        Empty DataFrame
        Columns: [type, remaining_days, quantity]
        Index: []
        >>> a = Food(5000).df
        >>> a
                         type  remaining_days  quantity
        0             staples               1  8.333333
        1             staples               2  8.333333
        2             staples               3  8.333333
        3             staples               4  8.333333
        4             staples               5  8.333333
        ..                ...             ...       ...
        739  packaged protein             176  6.944444
        740  packaged protein             177  6.944444
        741  packaged protein             178  6.944444
        742  packaged protein             179  6.944444
        743  packaged protein             180  6.944444
        <BLANKLINE>
        [744 rows x 3 columns]
        >>> a.equals(Food(a).df)
        True
        """
        if inventory is None:
            self.df = pd.DataFrame(columns=[
                "type",
                "remaining_days",
                "quantity"
            ]).astype(dtype={
                "type": str,
                "remaining_days": int,
                "quantity": float
            })
        elif isinstance(inventory, pd.DataFrame):
            self.df = inventory
        elif isinstance(inventory, (float, int)):
            type = []
            remaining_days = []
            quantity = []
            key_ls = []
            for c in TYPES.keys():
                if random.random() < 0.4:
                    key_ls.append(c)
            for t in key_ls:
                # Assume that the remaining shelf lives of foods are uniformly distributed within [1, max_days]
                max_days = random.randrange(1, 4)
                q = inventory * TYPES[t]["proportion"] / max_days
                for d in range(1, max_days + 1):
                    type.append(t)
                    remaining_days.append(d)
                    quantity.append(q)
            self.df = pd.DataFrame({"type": type, "remaining_days": remaining_days, "quantity": quantity})
        else:
            raise ValueError("Invalid input for initialization")
            #self.df = self.df.set_index(["type", "remaining_days"])


    @classmethod
    def generate_donation(cls, mean_total: float):
        """Generate food donation in a day. The quantity of each type and the total are all random, but their means are given
        :return:
        >>> food = Food.generate_donation(5000).df
        """
        type = []
        quantity = []
        remaining_days = []
        for t in TYPES.keys():
            # Assume that the remaining shelf lives of foods are uniformly distributed within [1, max_days]
            mean = mean_total * TYPES[t]["proportion"]
            low, high, stdev = 0.3 * mean, 5 * mean, 0.5 * mean
            beta = mod_beta_random(low, high, mean, stdev, 1).item()
            q = beta / TYPES[t]["max_days"]
            for d in range(1, TYPES[t]["max_days"] + 1):
                type.append(t)
                remaining_days.append(d)
                quantity.append(q)
        df = pd.DataFrame({"type": type, "remaining_days": remaining_days, "quantity": quantity})
        return Food(df)

    def sort_by_freshness(self, ascending=False):
        """
        Sort the food in each category by the remaining shelf life. Assume that clients prefer the freshest food,
        whereas food bank gives out food that is going to expire in order to reduce waste.
        :return:
        >>> a = Food(5000)
        >>> a.sort_by_freshness()
        >>> a.df
                                    type  remaining_days   quantity
        193  fresh fruits and vegetables              14  35.714286
        192  fresh fruits and vegetables              13  35.714286
        191  fresh fruits and vegetables              12  35.714286
        190  fresh fruits and vegetables              11  35.714286
        189  fresh fruits and vegetables              10  35.714286
        ..                           ...             ...        ...
        4                        staples               5   8.333333
        3                        staples               4   8.333333
        2                        staples               3   8.333333
        1                        staples               2   8.333333
        0                        staples               1   8.333333
        <BLANKLINE>
        [744 rows x 3 columns]
        """
        self.df = self.df.sort_values(by=["type", "remaining_days"], ascending=[True, ascending])

    def quality_control(self, num_days=1) -> Dict[str, float]:
        """ Subtract some days from the remaining shelf life of the food, remove the expired food from the inventory,
        and record the quantity of waste in each category.
        :param num_days: number of days since the last quality check
        :return: a dictionary storing the wasted food in each category
        >>> a = Food(5000)
        >>> sum(a.quality_control(float("inf")).values()) == 5000
        True
        >>> res = a.quality_control(20)
        >>> all([a.quality_control(10)[key] < value < a.quality_control(30)[key] for key, value in res.items()])
        True
        >>> a.quality_control(7)
        {FFV: 250.0, 'fresh protein': 350.0, 'packaged fruits and vegetables': 24.305555555555557, 'packaged protein': 48.611111111111114, 'staples': 58.333333333333336}
        """
        self.df["remaining_days"] -= num_days
        mask = self.df["remaining_days"] <= 0
        waste = self.df[mask]
        #waste_counter = waste.groupby(["type"])["quantity"].agg("sum")["quantity"].to_dict()
        waste_counter = waste.groupby(["type"])["quantity"].agg("sum").to_dict()
        self.df = self.df[~mask]
        return waste_counter

    def add(self, other):
        """ Add a new batch of food to inventory
        :param other:
        :return:

        """
        if isinstance(other, Food):
            other = other.df
        self.df = self.df.set_index(["type", "remaining_days"]).add(other.set_index(["type", "remaining_days"]),
                                                                    fill_value=0).reset_index()

    def subtract(self, other):
        """
        Subtract an existing batch of food from inventory
        :param other:
        :return:
        """
        if isinstance(other, Food):
            other = other.df
        self.df = self.df.set_index(["type", "remaining_days"]).sub(other.set_index(["type", "remaining_days"]),
                                                                    fill_value=0).reset_index()
        # if not valid: raise ValueError("Some food items does not exist or are not enough")

In [5]:
a = Food()
a.df

,type,remaining_days,quantity


In [35]:
b = Food(5000)
b.df

,type,remaining_days,quantity
0,fresh_fruits_and_vegetables,1,250.0
1,fresh_fruits_and_vegetables,2,250.0
2,fresh_protein,1,250.0
3,fresh_protein,2,250.0


In [ ]:
type(b.df["type"])

In [ ]:
a.subtract(b)
a.df

In [1]:
import pandas as pd
import numpy as np
from constants import *
from utils import *
#indexes = pd.MultiIndex.from_tuples([['paris']], names=['name'])
#columns = pd.MultiIndex.from_arrays([['route', 'action', 'action'], ['type', 'source', 'destination']], names=['first', 'second'])

#df = pd.DataFrame([[2, 3, 4]], index=indexes, columns=columns)

#print(df)

In [2]:
columns = [("num_people", ""), (STP, "demand"),(STP, "secured"),(STP, "lacking"),(STP, "purchased"),(FV, "demand"),(FV, "secured"),(FV, "lacking"),(FV, "purchased_fresh"),(FV, "purchased_packaged"),
           (PT, "demand"), (PT, "secured"),(PT, "lacking"),(PT, "purchased_fresh"),(PT, "purchased_packaged")]
df = pd.DataFrame(columns = pd.MultiIndex.from_tuples(columns))
#df = pd.DataFrame(columns = columns)
df

Empty DataFrame
Columns: [(num_people, ), (staples, demand), (staples, secured), (staples, lacking), (staples, purchased), (fruits_and_vegetables, demand), (fruits_and_vegetables, secured), (fruits_and_vegetables, lacking), (fruits_and_vegetables, purchased_fresh), (fruits_and_vegetables, purchased_packaged), (protein, demand), (protein, secured), (protein, lacking), (protein, purchased_fresh), (protein, purchased_packaged)]
Index: []

In [3]:
df.loc[:, (STP, "lacking")] = np.random.default_rng().normal(loc=0, scale=1, size=100)

In [4]:
df[(STP,"hhhh")] = 666
df

num_people staples                             fruits_and_vegetables  \
               demand secured   lacking purchased                demand   
0         NaN     NaN     NaN -0.794148       NaN                   NaN   
1         NaN     NaN     NaN -0.934035       NaN                   NaN   
2         NaN     NaN     NaN -0.172259       NaN                   NaN   
3         NaN     NaN     NaN -0.255360       NaN                   NaN   
4         NaN     NaN     NaN -0.253643       NaN                   NaN   
..        ...     ...     ...       ...       ...                   ...   
95        NaN     NaN     NaN  1.295553       NaN                   NaN   
96        NaN     NaN     NaN  0.805722       NaN                   NaN   
97        NaN     NaN     NaN -0.341367       NaN                   NaN   
98        NaN     NaN     NaN  1.152094       NaN                   NaN   
99        NaN     NaN     NaN  1.643735       NaN                   NaN   

                                                      protein                  \
   secured lacking purchased_fresh purchased_packaged  demand secured lacking   
0      NaN     NaN             NaN                NaN     NaN     NaN     NaN   
1      NaN     NaN             NaN                NaN     NaN     NaN     NaN   
2      NaN     NaN             NaN                NaN     NaN     NaN     NaN   
3      NaN     NaN             NaN                NaN     NaN     NaN     NaN   
4      NaN     NaN             NaN                NaN     NaN     NaN     NaN   
..     ...     ...             ...                ...     ...     ...     ...   
95     NaN     NaN             NaN                NaN     NaN     NaN     NaN   
96     NaN     NaN             NaN                NaN     NaN     NaN     NaN   
97     NaN     NaN             NaN                NaN     NaN     NaN     NaN   
98     NaN     NaN             NaN                NaN     NaN     NaN     NaN   
99     NaN     NaN             NaN                NaN     NaN     NaN     NaN   

                                      staples  
   purchased_fresh purchased_packaged    hhhh  
0              NaN                NaN     666  
1              NaN                NaN     666  
2              NaN                NaN     666  
3              NaN                NaN     666  
4              NaN                NaN     666  
..             ...                ...     ...  
95             NaN                NaN     666  
96             NaN                NaN     666  
97             NaN                NaN     666  
98             NaN                NaN     666  
99             NaN                NaN     666  

[100 rows x 16 columns]

In [2]:
rng = np.random.default_rng()
columns = [("num_people", ""), (STP, "total"), (STP, "base_secured"), (STP, "secured"), (STP, "demand"),
                   (STP, "purchased"),
                   (FV, "total"), (FV, "base_secured"), (FV, "secured"), (FV, "demand"), (FV, "demand_alt"),
                   (FV, "purchased_fresh"), (FV, "purchased_packaged"),
                   (PT, "total"), (PT, "base_secured"), (PT, "secured"), (PT, "demand"), (PT, "demand_alt"),
                   (PT, "purchased_fresh"), (PT, "purchased_packaged")]
clients = pd.DataFrame(columns=pd.MultiIndex.from_tuples(columns))
clients[("num_people", "")] = rng.choice(range(1, 11), 100, p=FAMILY_DISTRIBUTION)
for tp, value in PERSON_WEEKLY_DEMAND.items():
    mean, std = value["mean"], value["std"]
    low, high = 0.5 * mean, 2 * mean
    clients[(tp, "total")] = mod_beta_random(low, high, mean, std, 100) * clients[
        ("num_people", "")]

In [3]:
clients

num_people    staples                                        \
                   total base_secured secured demand purchased   
0           2  10.118659          NaN     NaN    NaN       NaN   
1           3  16.618000          NaN     NaN    NaN       NaN   
2           2  10.634335          NaN     NaN    NaN       NaN   
3           1   5.280491          NaN     NaN    NaN       NaN   
4           1   4.772332          NaN     NaN    NaN       NaN   
..        ...        ...          ...     ...    ...       ...   
95          2  10.424837          NaN     NaN    NaN       NaN   
96          2  10.831011          NaN     NaN    NaN       NaN   
97          1   4.853918          NaN     NaN    NaN       NaN   
98          1   5.710993          NaN     NaN    NaN       NaN   
99          2  11.116763          NaN     NaN    NaN       NaN   

   fruits_and_vegetables                                         \
                   total base_secured secured demand demand_alt   
0              25.421769          NaN     NaN    NaN        NaN   
1              36.312915          NaN     NaN    NaN        NaN   
2              26.975094          NaN     NaN    NaN        NaN   
3              11.667134          NaN     NaN    NaN        NaN   
4              12.866751          NaN     NaN    NaN        NaN   
..                   ...          ...     ...    ...        ...   
95             24.029030          NaN     NaN    NaN        NaN   
96             24.870935          NaN     NaN    NaN        NaN   
97             13.897343          NaN     NaN    NaN        NaN   
98             13.049427          NaN     NaN    NaN        NaN   
99             23.184371          NaN     NaN    NaN        NaN   

                                         protein                              \
   purchased_fresh purchased_packaged      total base_secured secured demand   
0              NaN                NaN  29.038465          NaN     NaN    NaN   
1              NaN                NaN  31.983297          NaN     NaN    NaN   
2              NaN                NaN  25.629375          NaN     NaN    NaN   
3              NaN                NaN  11.957561          NaN     NaN    NaN   
4              NaN                NaN  12.212004          NaN     NaN    NaN   
..             ...                ...        ...          ...     ...    ...   
95             NaN                NaN  24.249990          NaN     NaN    NaN   
96             NaN                NaN  24.015847          NaN     NaN    NaN   
97             NaN                NaN  12.775050          NaN     NaN    NaN   
98             NaN                NaN  12.965285          NaN     NaN    NaN   
99             NaN                NaN  23.709889          NaN     NaN    NaN   

                                                  
   demand_alt purchased_fresh purchased_packaged  
0         NaN             NaN                NaN  
1         NaN             NaN                NaN  
2         NaN             NaN                NaN  
3         NaN             NaN                NaN  
4         NaN             NaN                NaN  
..        ...             ...                ...  
95        NaN             NaN                NaN  
96        NaN             NaN                NaN  
97        NaN             NaN                NaN  
98        NaN             NaN                NaN  
99        NaN             NaN                NaN  

[100 rows x 20 columns]

In [42]:
clients

num_people    staples                                        \
                   total base_secured secured demand purchased   
0           1   4.784810     0.739065     NaN    NaN       NaN   
1           3  13.893051     0.657388     NaN    NaN       NaN   
2           4  18.743293     0.728415     NaN    NaN       NaN   
3           1   4.490979     0.546357     NaN    NaN       NaN   
4           5  25.975730     0.562824     NaN    NaN       NaN   
..        ...        ...          ...     ...    ...       ...   
95          2   8.995218     0.329766     NaN    NaN       NaN   
96          3  15.958354     0.636235     NaN    NaN       NaN   
97          2  10.770814     0.412172     NaN    NaN       NaN   
98          1   5.370961     0.769259     NaN    NaN       NaN   
99          1   5.221444     0.657785     NaN    NaN       NaN   

   fruits_and_vegetables                                         \
                   total base_secured secured demand demand_alt   
0              13.312765     0.664448     NaN    NaN        NaN   
1              43.192453     0.640523     NaN    NaN        NaN   
2              56.779748     0.336609     NaN    NaN        NaN   
3              14.338204     0.522459     NaN    NaN        NaN   
4              74.147372     0.593564     NaN    NaN        NaN   
..                   ...          ...     ...    ...        ...   
95             25.783559     0.750226     NaN    NaN        NaN   
96             41.845409     0.488774     NaN    NaN        NaN   
97             27.031331     0.475196     NaN    NaN        NaN   
98             12.733990     0.363060     NaN    NaN        NaN   
99             12.483846     0.480230     NaN    NaN        NaN   

                                         protein                              \
   purchased_fresh purchased_packaged      total base_secured secured demand   
0              NaN                NaN  11.797404     0.480223     NaN    NaN   
1              NaN                NaN  39.999795     0.707331     NaN    NaN   
2              NaN                NaN  51.375704     0.383128     NaN    NaN   
3              NaN                NaN  14.353721     0.615827     NaN    NaN   
4              NaN                NaN  67.321619     0.398867     NaN    NaN   
..             ...                ...        ...          ...     ...    ...   
95             NaN                NaN  22.823105     0.485960     NaN    NaN   
96             NaN                NaN  35.818854     0.751570     NaN    NaN   
97             NaN                NaN  26.235245     0.348593     NaN    NaN   
98             NaN                NaN  10.818363     0.405680     NaN    NaN   
99             NaN                NaN  13.164161     0.671345     NaN    NaN   

                                                  
   demand_alt purchased_fresh purchased_packaged  
0         NaN             NaN                NaN  
1         NaN             NaN                NaN  
2         NaN             NaN                NaN  
3         NaN             NaN                NaN  
4         NaN             NaN                NaN  
..        ...             ...                ...  
95        NaN             NaN                NaN  
96        NaN             NaN                NaN  
97        NaN             NaN                NaN  
98        NaN             NaN                NaN  
99        NaN             NaN                NaN  

[100 rows x 20 columns]

In [43]:
clients.loc[:, (slice(None), ['purchased', 'purchased_fresh', 'purchased_packaged'])] = 0.
clients

C:\Users\techt\AppData\Local\Temp/ipykernel_9852/3023708085.py:1: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  clients.loc[:, (slice(None), ['purchased', 'purchased_fresh', 'purchased_packaged'])] = 0.


num_people    staples                                        \
                   total base_secured secured demand purchased   
0           1   4.784810     0.739065     NaN    NaN       0.0   
1           3  13.893051     0.657388     NaN    NaN       0.0   
2           4  18.743293     0.728415     NaN    NaN       0.0   
3           1   4.490979     0.546357     NaN    NaN       0.0   
4           5  25.975730     0.562824     NaN    NaN       0.0   
..        ...        ...          ...     ...    ...       ...   
95          2   8.995218     0.329766     NaN    NaN       0.0   
96          3  15.958354     0.636235     NaN    NaN       0.0   
97          2  10.770814     0.412172     NaN    NaN       0.0   
98          1   5.370961     0.769259     NaN    NaN       0.0   
99          1   5.221444     0.657785     NaN    NaN       0.0   

   fruits_and_vegetables                                         \
                   total base_secured secured demand demand_alt   
0              13.312765     0.664448     NaN    NaN        NaN   
1              43.192453     0.640523     NaN    NaN        NaN   
2              56.779748     0.336609     NaN    NaN        NaN   
3              14.338204     0.522459     NaN    NaN        NaN   
4              74.147372     0.593564     NaN    NaN        NaN   
..                   ...          ...     ...    ...        ...   
95             25.783559     0.750226     NaN    NaN        NaN   
96             41.845409     0.488774     NaN    NaN        NaN   
97             27.031331     0.475196     NaN    NaN        NaN   
98             12.733990     0.363060     NaN    NaN        NaN   
99             12.483846     0.480230     NaN    NaN        NaN   

                                         protein                              \
   purchased_fresh purchased_packaged      total base_secured secured demand   
0              0.0                0.0  11.797404     0.480223     NaN    NaN   
1              0.0                0.0  39.999795     0.707331     NaN    NaN   
2              0.0                0.0  51.375704     0.383128     NaN    NaN   
3              0.0                0.0  14.353721     0.615827     NaN    NaN   
4              0.0                0.0  67.321619     0.398867     NaN    NaN   
..             ...                ...        ...          ...     ...    ...   
95             0.0                0.0  22.823105     0.485960     NaN    NaN   
96             0.0                0.0  35.818854     0.751570     NaN    NaN   
97             0.0                0.0  26.235245     0.348593     NaN    NaN   
98             0.0                0.0  10.818363     0.405680     NaN    NaN   
99             0.0                0.0  13.164161     0.671345     NaN    NaN   

                                                  
   demand_alt purchased_fresh purchased_packaged  
0         NaN             0.0                0.0  
1         NaN             0.0                0.0  
2         NaN             0.0                0.0  
3         NaN             0.0                0.0  
4         NaN             0.0                0.0  
..        ...             ...                ...  
95        NaN             0.0                0.0  
96        NaN             0.0                0.0  
97        NaN             0.0                0.0  
98        NaN             0.0                0.0  
99        NaN             0.0                0.0  

[100 rows x 20 columns]

In [41]:
clients.loc[:, (slice(None),"base_secured")] = rng.uniform(0.3, 0.8, (100, 3))

C:\Users\techt\AppData\Local\Temp/ipykernel_9852/1441275944.py:1: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  clients.loc[:, (slice(None),"base_secured")] = rng.uniform(0.3, 0.8, (100, 3))


In [28]:
clients[clients.columns[(slice(None),"secured")]] = rng.uniform(0.3, 0.8, (100, 3))
clients

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [12]:
df.loc[:0, (STP, "secured")]

0    NaN
Name: (staples, secured), dtype: object

In [15]:
rng = np.random.default_rng()

In [13]:
np.random.default_rng().integers(0, 7)

2

In [20]:
rng.choice(range(1, 11), 100, p=FAMILY_DISTRIBUTION)

array([1, 2, 4, 1, 2, 5, 6, 1, 6, 2, 2, 2, 2, 2, 4, 2, 2, 1, 2, 2, 2, 2,
       1, 3, 2, 2, 1, 4, 3, 2, 1, 2, 1, 4, 2, 2, 3, 2, 2, 3, 5, 2, 2, 3,
       3, 1, 6, 4, 4, 2, 3, 4, 2, 1, 3, 1, 2, 2, 1, 2, 5, 5, 2, 3, 4, 1,
       1, 1, 1, 2, 3, 2, 2, 2, 2, 9, 2, 3, 1, 2, 3, 1, 2, 3, 2, 4, 2, 2,
       5, 4, 2, 8, 5, 5, 1, 1, 2, 2, 5, 1])

In [23]:
a: dict

In [24]:
a

NameError: name 'a' is not defined

In [26]:
b=rng.uniform(0.3, 0.8, (10, 3))
b

array([[0.46713053, 0.61680935, 0.78574576],
       [0.32584138, 0.41957676, 0.638874  ],
       [0.54477672, 0.60752674, 0.42128631],
       [0.77053442, 0.4716277 , 0.73892086],
       [0.73084925, 0.40296935, 0.58021526],
       [0.71783257, 0.45156034, 0.58151702],
       [0.41437203, 0.5648428 , 0.39816521],
       [0.72415256, 0.49839417, 0.78198777],
       [0.59630888, 0.4178616 , 0.68102478],
       [0.78640936, 0.52372774, 0.69857244]])

In [28]:
b

array([[0.46713053, 0.61680935, 0.78574576],
       [0.32584138, 0.41957676, 0.638874  ],
       [0.54477672, 0.60752674, 0.42128631],
       [0.77053442, 0.4716277 , 0.73892086],
       [0.73084925, 0.40296935, 0.58021526],
       [0.71783257, 0.45156034, 0.58151702],
       [0.41437203, 0.5648428 , 0.39816521],
       [0.72415256, 0.49839417, 0.78198777],
       [0.59630888, 0.4178616 , 0.68102478],
       [0.78640936, 0.52372774, 0.69857244]])

In [29]:
b.sample(frac=1)

AttributeError: 'numpy.ndarray' object has no attribute 'sample'

In [36]:
from utils import Food

In [11]:
df = Food(5000).df
df

,type,remaining_days,quantity
0,fresh_fruits_and_vegetables,1,500.000000
1,packaged_protein,1,416.666667
2,packaged_protein,2,416.666667
3,packaged_protein,3,416.666667


In [14]:
1e-7

1e-07

In [13]:
df[df["remaining_days"]==1].reset_index(drop=True)

,type,remaining_days,quantity
0,fresh_fruits_and_vegetables,1,500.000000
1,packaged_protein,1,416.666667
